In [50]:
import wandb
import pandas as pd
import os
from tqdm import tqdm

# from table_plotter import print_result_table

In [51]:
api = wandb.Api(timeout=600)


In [52]:
# Specify cache directory
cache_dir = "./wandb_cache"
os.makedirs(cache_dir, exist_ok=True)

In [53]:
dfs = []
skipped_runs = []  # List to store IDs of skipped runs

evaluation_keys = ['Evaluation/acc_imp_perc', 'Evaluation/exist_imp_perc', 'Evaluation/reach_imp_perc', 'Evaluation/path_length',
                   'Evaluation/fn_imp_perc', 'Evaluation/fp_imp_perc', 'Evaluation/tn_imp_perc', 'Evaluation/tp_imp_perc', 
                   'Evaluation/solvability', 'Evaluation/playability']
                     

In [54]:
def get_dataframe_from_run(run):
    
    for run in tqdm(runs):
    
        # Define cache filename based on run ID
        cache_file = os.path.join(cache_dir, f"{run.id}.csv")
        
        # Check if cached file exists
        if os.path.exists(cache_file):
            # Load cached DataFrame
            df = pd.read_csv(cache_file)
        else:
            if run.state == "running":
                print(f"Skipping run ID: {run.id} (state: {run.state})")
                continue
            
            df = run.history(keys=["Evaluation/llm_iteration", *evaluation_keys[:1]])
    
            def append_key(src_df, key):
    
                tgt_df = run.history(keys=[key, "Evaluation/llm_iteration"])
                src_df = pd.merge(src_df, tgt_df, on="Evaluation/llm_iteration", how="outer")
                src_df = src_df.drop(columns=["_step_x", "_step_y"], errors="ignore")
                return src_df
    
            for key in evaluation_keys[1:]:
                try:
                    df = append_key(df, key)
                except Exception as e:
                    print(f"Error: {e} at run ID: {run.id}")
    
            
            # Add run config to DataFrame with prefix 'config.'
            for key, value in run.config.items():
                if isinstance(value, list):
                    value = ",".join(map(str, value))  # Convert list to comma-separated string
                df[key] = value
    
            # 기본값 설정
            default_values = {'n_self_alignment': 0, 'feedback_type': 'default'}
            # 열이 없을 경우 기본값으로 채워 넣기
            for col, value in default_values.items():
                if col not in df.columns:
                    df[col] = value
            
             
            # Filter columns
            key_filter = ['run_id', 'final_state', 'target_character', 'pe', 'gpt_model', 'branch_factor', 'exp_name', 'evaluator', 'total_iterations', 'n_self_alignment', 'feedback_type', 'feedback_input_type', 'total_timesteps', 
                          'reward_feature', 'fewshot', 'problem', 'seed', 'n_aux_best', 'n_aux_worst',
                          'Evaluation/llm_iteration'] + evaluation_keys
            auxiliary_key_filter = []
            
            df['run_id'] = run.id  # Add run ID as a column
            df['final_state'] = run.state
            
            try:
                df = df[key_filter + auxiliary_key_filter]
            except KeyError:
                df = df[key_filter]
            
            # Save DataFrame to cache as CSV
            df.to_csv(cache_file, index=False)
        
        dfs.append(df)
    
    # Concatenate all DataFrames
    df = pd.concat(dfs, ignore_index=True)
    
    return df

In [55]:
runs = api.runs("inchangbaek4907/scenario-aux")
scenario_df = get_dataframe_from_run(runs)
scenario_df

  0%|          | 0/55 [00:00<?, ?it/s]

Skipping run ID: pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b0w2_chr-1_1_s-2 (state: running)
Skipping run ID: pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b2w1_chr-2_1_s-3 (state: running)
Skipping run ID: pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b2w1_chr-2_1_s-5 (state: running)
Skipping run ID: pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b2w1_chr-1_1_s-6 (state: running)
Skipping run ID: pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b0w2_chr-1_1_s-1 (state: running)
Skipping run ID: pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b2w1_chr-1_1_s-5 (state: running)
Skipping run ID: pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b2w1_chr-2_1_s-4 (state: running)
Skipping run ID: pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b2w1_chr-1_1_s-4 (state: running)
Skipping run ID: pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b0w2_chr-2_1_s-1 (state: running)


100%|██████████| 55/55 [00:00<00:00, 164.14it/s]

Skipping run ID: pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b0w2_chr-2_1_s-2 (state: running)
Skipping run ID: pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b0w2_chr-1_1_s-3 (state: running)
Skipping run ID: pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b0w2_chr-2_1_s-3 (state: running)
Skipping run ID: pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b0w2_chr-1_1_s-4 (state: running)
Skipping run ID: pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b0w2_chr-2_1_s-4 (state: running)


,run_id,final_state,target_character,pe,gpt_model,branch_factor,exp_name,evaluator,total_iterations,n_self_alignment,...,Evaluation/acc_imp_perc,Evaluation/exist_imp_perc,Evaluation/reach_imp_perc,Evaluation/path_length,Evaluation/fn_imp_perc,Evaluation/fp_imp_perc,Evaluation/tn_imp_perc,Evaluation/tp_imp_perc,Evaluation/solvability,Evaluation/playability
0,pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b1w0_chr-...,finished,1,got,gpt-4o,2,aux,hr,6,0,...,0.011111,0.533333,0.533333,26.000000,2.700000,0.266667,0.000000,0.033333,0.133333,0.966667
1,pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b1w0_chr-...,finished,2,got,gpt-4o,2,aux,hr,6,0,...,0.000000,0.633333,0.166667,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b1w0_chr-...,finished,2,got,gpt-4o,2,aux,hr,6,0,...,0.000000,0.666667,0.016667,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b1w0_chr-...,finished,2,got,gpt-4o,2,aux,hr,6,0,...,0.033333,0.716667,0.250000,26.000000,2.900000,0.000000,0.033333,0.066667,0.033333,0.033333
4,pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b1w0_chr-...,finished,2,got,gpt-4o,2,aux,hr,6,0,...,0.066667,0.400000,0.250000,26.900000,2.800000,0.000000,0.133333,0.066667,0.133333,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b2w1_chr-...,finished,1,got,gpt-4o,2,aux,hr,6,0,...,0.011111,0.533333,0.433333,26.222223,2.833333,0.133333,0.000000,0.033333,0.066667,0.900000
224,pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b2w1_chr-...,finished,1,got,gpt-4o,2,aux,hr,6,0,...,0.011111,0.566667,0.500000,26.545456,2.900000,0.066667,0.000000,0.033333,0.033333,0.733333
225,pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b2w1_chr-...,finished,1,got,gpt-4o,2,aux,hr,6,0,...,0.011111,0.533333,0.533333,26.000000,2.700000,0.266667,0.000000,0.033333,0.133333,0.966667
226,pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b2w1_chr-...,finished,1,got,gpt-4o,2,aux,hr,6,0,...,0.011111,0.500000,0.500000,26.000000,2.633333,0.333333,0.000000,0.033333,0.166667,0.966667


In [56]:
# Print summary of skipped runs
print("\nSummary of Skipped Runs:")
print(f"Total skipped runs: {len(skipped_runs)}")
print("Skipped run IDs:", skipped_runs)


Summary of Skipped Runs:
Total skipped runs: 0
Skipped run IDs: []


In [57]:
df = pd.concat([scenario_df], ignore_index=True)

In [58]:
# only the target_character is '1' or '2'
df = df[df['target_character'].isin([1, 2])]
df

,run_id,final_state,target_character,pe,gpt_model,branch_factor,exp_name,evaluator,total_iterations,n_self_alignment,...,Evaluation/acc_imp_perc,Evaluation/exist_imp_perc,Evaluation/reach_imp_perc,Evaluation/path_length,Evaluation/fn_imp_perc,Evaluation/fp_imp_perc,Evaluation/tn_imp_perc,Evaluation/tp_imp_perc,Evaluation/solvability,Evaluation/playability
0,pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b1w0_chr-...,finished,1,got,gpt-4o,2,aux,hr,6,0,...,0.011111,0.533333,0.533333,26.000000,2.700000,0.266667,0.000000,0.033333,0.133333,0.966667
1,pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b1w0_chr-...,finished,2,got,gpt-4o,2,aux,hr,6,0,...,0.000000,0.633333,0.166667,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b1w0_chr-...,finished,2,got,gpt-4o,2,aux,hr,6,0,...,0.000000,0.666667,0.016667,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b1w0_chr-...,finished,2,got,gpt-4o,2,aux,hr,6,0,...,0.033333,0.716667,0.250000,26.000000,2.900000,0.000000,0.033333,0.066667,0.033333,0.033333
4,pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b1w0_chr-...,finished,2,got,gpt-4o,2,aux,hr,6,0,...,0.066667,0.400000,0.250000,26.900000,2.800000,0.000000,0.133333,0.066667,0.133333,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b2w1_chr-...,finished,1,got,gpt-4o,2,aux,hr,6,0,...,0.011111,0.533333,0.433333,26.222223,2.833333,0.133333,0.000000,0.033333,0.066667,0.900000
224,pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b2w1_chr-...,finished,1,got,gpt-4o,2,aux,hr,6,0,...,0.011111,0.566667,0.500000,26.545456,2.900000,0.066667,0.000000,0.033333,0.033333,0.733333
225,pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b2w1_chr-...,finished,1,got,gpt-4o,2,aux,hr,6,0,...,0.011111,0.533333,0.533333,26.000000,2.700000,0.266667,0.000000,0.033333,0.133333,0.966667
226,pe-got_it-6_fit-hr_exp-aux_t-sce_aux-b2w1_chr-...,finished,1,got,gpt-4o,2,aux,hr,6,0,...,0.011111,0.500000,0.500000,26.000000,2.633333,0.333333,0.000000,0.033333,0.166667,0.966667


In [59]:
time_str = pd.Timestamp.now().strftime("%Y-%m-%d-%H-%M-%S")

In [60]:
df.to_csv(f"wandb_aux_output_{time_str}.csv", index=False)